In [136]:
import copy

def bfs(start, valves):
    state = {
        'node': start,
        'flow': 0,
        'cost': 0,
        'cost_eq': '',
        'minute': 1,
        'opened': []
    }
    queue = [state]
    count = 0
    width = 10000
    last_minute = 0
    limit = 31
    max_state = {}
    while(len(queue) > 0):
        count += 1
        state = queue.pop(0)
        if(state['minute'] > limit):
            if(max_state == {}):
                max_state = state
            elif(state['cost'] > max_state['cost']):
                max_state = state
            continue
        if(state['minute'] > last_minute):
            last_minute = state['minute']
            queue = sorted(queue, key=lambda k: k['cost'], reverse=True)
            queue = queue[:width]
            print(f"Minute: {str(state['minute'])} States to check: {str(len(queue))}")
            # if(len(queue)):
            #     print(f"Minute: {str(state['minute'])} States to check: {str(len(queue))} Max cost: {str(queue[0]['cost'])} Max flow: {str(queue[0]['flow'])} Max path: {str(queue[0]['path'])}")
        # print(state)
        neighbors = valves[state['node']]['tunnels']
        neighbors = sorted(neighbors, key=lambda k: valves[k]['flow'], reverse=True)
        flow = valves[state['node']]['flow']
        opened = state['opened']
        if(flow and state['node'] not in opened):
            new_state = copy.deepcopy(state)
            new_state['minute'] += 1
            new_state['cost'] += flow * (limit - new_state['minute'])
            new_state['cost_eq'] += f"{flow} * {(limit - new_state['minute'])} + "
            new_state['flow'] += flow
            new_state['opened'].append(state['node'])
            # new_state['path'].append((state['node'], new_state['minute'], "opened"))
            queue.append(new_state)
        for neighbor in neighbors:
            new_state = copy.deepcopy(state)
            new_state['node'] = neighbor
            new_state['minute'] += 1
            # new_state['path'].append((neighbor, new_state['minute'], "moved"))
            queue.append(new_state)
    print(max_state)

In [137]:
import re
with open('input.txt', 'r') as f:
    input = f.read()
input = input.splitlines()
regex = re.compile(r'Valve (.*) has flow rate=(\d+); tunnel(?:s?) lead(?:s?) to valve(?:s?) (.*)')
valves = {}
for line in input:
    valve, flow, tunnels = regex.match(line).groups()
    valves[valve] = {'flow': int(flow), 'tunnels': tunnels.split(', ')}
# print(valves)
answer = bfs('AA', valves)
# print(answer)

Minute: 1 States to check: 0
Minute: 2 States to check: 2
Minute: 3 States to check: 8
Minute: 4 States to check: 24
Minute: 5 States to check: 70
Minute: 6 States to check: 197
Minute: 7 States to check: 555
Minute: 8 States to check: 1532
Minute: 9 States to check: 4246
Minute: 10 States to check: 10000
Minute: 11 States to check: 10000
Minute: 12 States to check: 10000
Minute: 13 States to check: 10000
Minute: 14 States to check: 10000
Minute: 15 States to check: 10000
Minute: 16 States to check: 10000
Minute: 17 States to check: 10000
Minute: 18 States to check: 10000
Minute: 19 States to check: 10000
Minute: 20 States to check: 10000
Minute: 21 States to check: 10000
Minute: 22 States to check: 10000
Minute: 23 States to check: 10000
Minute: 24 States to check: 10000
Minute: 25 States to check: 10000
Minute: 26 States to check: 10000
Minute: 27 States to check: 10000
Minute: 28 States to check: 10000
Minute: 29 States to check: 10000
Minute: 30 States to check: 10000
Minute: 31 St

In [502]:
import copy

def bfs_with_elephant(start, valves):
    state = {
        'node': start,
        'elephant_node': start,
        'flow': 0,
        'cost': 0,
        'sort_cost': 0,
        'new_sort_cost': 0,
        'cost_eq': '',
        'minute': 1,
        'opened': {},
    }
    queue = [state]
    visited_count = 0
    width = 10000
    last_minute = 0
    limit = 4
    max_state = {}
    visited = {}
    while(len(queue) > 0):
        state = queue.pop(0)
        if(state['minute'] > limit or len(state['opened']) == len(valves)):
            if(max_state == {}):
                max_state = state
            elif(state['cost'] > max_state['cost']):
                max_state = state
            continue
        if(state['minute'] > last_minute):
            last_minute = state['minute']
            queue = sorted(queue, key=lambda k: k['new_sort_cost'], reverse=True)
            # print(queue)
            # print()
            queue = queue[:width]
            if(len(queue)):
                print(f"Minute: {str(state['minute'])} States to check: {str(len(queue))} Max Flow: {str(queue[0]['flow'])} Max Cost: {str(queue[0]['cost'])}")
        neighbors = valves[state['node']]['tunnels']
        neighbors = sorted(neighbors, key=lambda k: valves[k]['flow'], reverse=True)
        opened = state['opened'].keys()
        state['cost'] = sum([valves[k]['flow'] * (state['minute'] - v + 1) for k,v in state['opened'].items()])
        for neighbor in neighbors:
            for neighbor2 in neighbors:
                if neighbor2 == neighbor:
                    continue
                new_state = copy.deepcopy(state)
                new_state['node'] = neighbor
                new_state['elephant_node'] = neighbor2
                new_state['minute'] += 1
                key = f"{','.join(new_state['opened'].keys())}_{new_state['node']}_{new_state['elephant_node']}_{new_state['new_sort_cost']}_{new_state['minute']}"
                if(key in visited.keys()):
                    visited_count += 1
                else:
                    visited[key] = True
                    queue.append(new_state)
        if((valves[state['node']]['flow'] or valves[state['elephant_node']]['flow']) and (state['node'] not in opened or state['elephant_node'] not in opened)):
            new_state = copy.deepcopy(state)
            new_state['minute'] += 1
            move = False
            if(valves[state['node']]['flow'] != 0 and state['node'] not in opened):
                flow = valves[state['node']]['flow']
                new_state['cost_eq'] += f"{flow} * {(limit - new_state['minute'])} + "
                new_state['flow'] += flow
                new_state['opened'][state['node']] = new_state['minute']
            else:
                move = ''
            if(valves[state['elephant_node']]['flow'] != 0 and state['elephant_node'] not in opened):
                flow = valves[state['elephant_node']]['flow']
                new_state['cost_eq'] += f"{flow} * {(limit - new_state['minute'])} + "
                new_state['flow'] += flow
                new_state['opened'][state['elephant_node']] = new_state['minute']
            else:
                move = 'elephant_'
            sort_cost = [valves[x]['flow'] for x in valves.keys() if x not in new_state['opened']]
            sort_cost = sorted(sort_cost, reverse=False)
            new_state['sort_cost'] = ((limit - state['minute']) * flow) + sum([x * i for i, x in enumerate(sort_cost)])
            new_state['new_sort_cost'] = sum([valves[k]['flow'] * (limit - v) for k,v in new_state['opened'].items()]) + sum([x * (limit - state['minute'] - i) for i, x in enumerate(sort_cost)])
            print(new_state)
            if(move != False):
                for neighbor in neighbors:
                    new_state = copy.deepcopy(new_state)
                    new_state[f'{move}node'] = neighbor
                    key = f"{','.join(new_state['opened'].keys())}_{new_state['node']}_{new_state['elephant_node']}_{new_state['new_sort_cost']}_{new_state['minute']}"
                    if(key in visited.keys()):
                        visited_count += 1
                    else:
                        visited[key] = True
                        queue.append(new_state)
            else:
                key = f"{','.join(new_state['opened'].keys())}_{new_state['node']}_{new_state['elephant_node']}_{new_state['new_sort_cost']}_{new_state['minute']}"
                if(key in visited.keys()):
                    visited_count += 1
                else:
                    visited[key] = True
                    queue.append(new_state)
    print(visited_count)
    print(max_state)

In [503]:
import re
with open('input.txt', 'r') as f:
    input = f.read()
input = input.splitlines()
regex = re.compile(r'Valve (.*) has flow rate=(\d+); tunnel(?:s?) lead(?:s?) to valve(?:s?) (.*)')
valves = {}
for line in input:
    valve, flow, tunnels = regex.match(line).groups()
    valves[valve] = {'flow': int(flow), 'tunnels': tunnels.split(', ')}
# print(valves)
answer = bfs_with_elephant('AA', valves)
# print(answer)

Minute: 2 States to check: 5 Max Flow: 0 Max Cost: 0
{'node': 'DD', 'elephant_node': 'BB', 'flow': 33, 'cost': 0, 'sort_cost': 329, 'new_sort_cost': -174, 'cost_eq': '20 * 1 + 13 * 1 + ', 'minute': 3, 'opened': {'DD': 3, 'BB': 3}}
{'node': 'DD', 'elephant_node': 'II', 'flow': 20, 'cost': 0, 'sort_cost': 464, 'new_sort_cost': -282, 'cost_eq': '20 * 1 + ', 'minute': 3, 'opened': {'DD': 3}}
{'node': 'BB', 'elephant_node': 'DD', 'flow': 33, 'cost': 0, 'sort_cost': 343, 'new_sort_cost': -174, 'cost_eq': '13 * 1 + 20 * 1 + ', 'minute': 3, 'opened': {'BB': 3, 'DD': 3}}
{'node': 'BB', 'elephant_node': 'II', 'flow': 13, 'cost': 0, 'sort_cost': 492, 'new_sort_cost': -317, 'cost_eq': '13 * 1 + ', 'minute': 3, 'opened': {'BB': 3}}
{'node': 'II', 'elephant_node': 'DD', 'flow': 20, 'cost': 0, 'sort_cost': 464, 'new_sort_cost': -282, 'cost_eq': '20 * 1 + ', 'minute': 3, 'opened': {'DD': 3}}
{'node': 'II', 'elephant_node': 'BB', 'flow': 13, 'cost': 0, 'sort_cost': 492, 'new_sort_cost': -317, 'cost_eq'

In [ ]:
20 * 24 + 22 * 23 + 13 * 22